In [2]:
import torch
import torch.nn as nn
from torch.autograd import Variable
from transformer import *
from util import *
from data import *

In [11]:
train, valid, test = get_dis('/home/yuhuiz/Transformer/data/csu/', 'csu_bpe', 'csu', 600, False)
text_encoder = TextEncoder('/home/yuhuiz/Transformer/data/sage/encoder_bpe_50000.json', '/home/yuhuiz/Transformer/data/sage/vocab_50000.bpe')
encoder = text_encoder.encoder
encoder['_pad_'] = len(encoder)
encoder['_start_'] = len(encoder)
encoder['_end_'] = len(encoder)
encoder['_unk_'] = len(encoder)

for split in ['s1']:
    for data_type in ['train', 'valid', 'test']:
        num_sents = []
        y_sents = []
        for sent in eval(data_type)[split]:
            num_sent = text_encoder.encode([sent], lazy=True, bpe=False)[0]
            num_sents.append([encoder['_start_']] + num_sent + [encoder['_end_']])
        eval(data_type)[split] = np.array(num_sents)

In [23]:
model = torch.load('/home/yuhuiz/Transformer/exp/bpe/transformer_auxiliary_pretrain/model-8.pickle', map_location='cpu')

In [18]:
print train['s1']

[list([49671, 25263, 9, 4, 87, 80, 110, 323, 359, 1751, 1288, 52, 5, 3, 985, 102, 10, 4, 138, 8, 2847, 996, 2, 1056, 1779, 0, 20, 21, 48, 138, 11, 17992, 18, 26, 376, 8, 513, 997, 1658, 17, 1978, 3014, 15, 2, 1056, 1779, 17, 53, 306, 515, 15, 0, 64, 3, 48, 138, 1, 25263, 12, 217, 5, 46, 1074, 17, 4293, 20, 1864, 5078, 62, 15, 78, 207, 5, 555, 35, 58, 867, 6143, 5, 25263, 38, 366, 151, 0, 18, 97, 1243, 3232, 11, 21, 319, 2, 3531, 2, 1531, 2, 1834, 20, 21, 5842, 0, 18, 55, 484, 5, 46, 2148, 2803, 11, 21, 5842, 2, 1521, 357, 78, 9, 2689, 2, 97, 4735, 517, 0, 25263, 38, 777, 46, 2897, 248, 64, 593, 3, 306, 29, 31, 18, 97, 517, 3748, 21, 453, 0, 9591, 55, 1981, 22, 18, 207, 1979, 7065, 8, 1854, 32, 23, 18, 12, 33, 228, 788, 0, 9591, 2122, 22, 58, 9, 821, 5, 8961, 25263, 35, 918, 35, 9, 182, 2, 22, 25263, 2280, 4679, 11, 397, 0, 25263, 146, 607, 2608, 2068, 1297, 180, 296, 33, 29, 6, 1030, 1163, 30, 116, 244, 1, 116, 229, 16644, 32929, 6, 3860, 6, 1139, 17, 34277, 15, 88, 17, 48, 606, 30, 91

In [65]:
s1 = test['s1']
target = test['label']
idx = 4

s1_batch = pad_batch(s1[idx:idx+1], encoder['_pad_'])
label_batch = target[idx:idx+1]

b = Batch(s1_batch, label_batch, [], encoder['_pad_'])
model.config['metamap'] = False
clf_output = model(b, clf=True, lm=False)
pred = (torch.sigmoid(clf_output) > 0.5).data.numpy().astype(float)

print clf_output
print label_batch
print pred

tensor([[-13.1561, -14.6234, -14.4348, -14.9629, -14.7689, -12.6590,
         -11.6367, -10.6716,  -7.7468,  -9.1299,  -8.6858,  -8.6477,
          -8.7555,  -7.9244,  -7.2333,  -6.3520,  -7.3136,  -5.8155,
          -7.9268, -12.8125,  -5.9897,  -5.6969,  -7.8255,  -7.4285,
          -2.8825,  -7.4164,  -6.6447,  -4.0579,   5.7133,  -7.2892,
          -6.5232,  -6.2227,  -5.4964,  -5.7454,  -6.1864,  -6.4600,
          -5.3494,  -5.5180,  -5.0549,  -2.9926,   5.1993,   6.0656]])
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1.]]
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1.]]


In [76]:
y = clf_output[0][41]
x = model.tgt_embed(b.s1)
print y
print x

tensor(6.0656)
tensor([[[   0.0697,    0.3765,   -0.0238,  ...,    0.9109,
            -0.1527,    0.4537],
         [   1.0225,    1.6396,    2.2117,  ...,    2.1428,
             0.0000,    0.2211],
         [   1.1746,   -0.5720,    1.3074,  ...,    0.4033,
             0.2824,    0.5981],
         ...,
         [  -0.2872,   -0.0000,    2.7038,  ...,    1.1355,
             0.1987,    1.2120],
         [  -1.0221,    1.0823,   -1.1686,  ...,    0.0446,
            -0.7870,    1.3837],
         [  -1.7926,    0.1317,    1.2507,  ...,    5.8325,
             3.3553,   -0.0000]]])
True


In [78]:
model.zero_grad()
# grad = torch.ones(x.shape)
# torch.autograd.grad(y, x, grad_outputs=grad, retain_graph=True, allow_unused=True)
y.backward(retain_graph=True)

In [87]:
print model.tgt_embed[0]

AttributeError: 'Embeddings' object has no attribute 'weight'